In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
url = 'https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fs3-ap-northeast-1.amazonaws.com%2Fpsh-ex-ftnikkei-3937bb4%2Fimages%2F1%2F1%2F2%2F8%2F14018211-7-eng-GB%2F0529N-Solar.jpg?source=nar-cms'
Image(url= url, width=600, height=600, unconfined=True)


    

Image Source:-www.asia.nikkei.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import files

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
#import n2w
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt

In [ ]:
# load data related to plant# 1

df_gen=pd.read_csv('../input/solar-power-generation-data/Plant_1_Generation_Data.csv')
df_weather=pd.read_csv('../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')

# Install n2w library

In [ ]:
pip install n2w

In [ ]:
import n2w

In [ ]:
df_gen.head()

In [ ]:
df_weather.head()

## Data Cleaning

In [ ]:
# drop unwanted columns. 
df_weather = df_weather.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_gen = df_gen.drop(['PLANT_ID'], axis=1)

# convert the date_time column into timedate stamp
df_gen["DATE_TIME"] = df_gen["DATE_TIME"].apply(pd.to_datetime)
df_weather["DATE_TIME"] = df_weather["DATE_TIME"].apply(pd.to_datetime)

In [ ]:
# Convert the sourcekey to meaningful representation. 
num = list(np.arange(0,22))

#extract source key from df_gen file. 
source_key = list(df_gen['SOURCE_KEY'].unique())

source_num = []
for n in num:
    source_num.append(n2w.convert(n))

#Replace source_key list in dataframe by number in words. 
for n in range(len(source_key)):
    df_gen = df_gen.replace(source_key[n],source_num[n])  


In [ ]:
df_gen.head()

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/PV-system-schematics-residential-Eng.png/360px-PV-system-schematics-residential-Eng.png'
Image(url= url, width=600, height=600, unconfined=True)

image source wikimedia.org

In [ ]:
## Group data based on inverter. 

df_inv_id = {k: v for k, v in df_gen.groupby('SOURCE_KEY')}

## Assumptions

1. Sun raise and sunset falls between 05.30 and 19.30, for all the places in India. The power generation happens during 
these hours. (#https://www.timeanddate.com/astronomy/india)
2. Each record should have 14hours * 60min/15min = 56 +1 = 57 rows for weather and gen df's. (time slot:- 05.30 and 19.30)
3. All the inverters are located almost at same location. The weather readings based of one sensor represents all other sets of inverters
4. Solar cell performance hinders based on clouds blocking sun rays. The weather data provided has only ambient temp readings/ module readings does not provide details of preciptation amount, Direct radition, diffuse radition, Total Cloud cover, low, mid and high cloud cover. (https://www.meteoblue.com/en/weather/archive/export/india_el-salvador_3585481)
5. Total Power Output = Total Area x Solar Irradiance x Conversion Efficiency. The total area of the solar cells is unknown, conversion efficiency is unknown. The power unit of DC Power is kW, solar irradiance unit is not provided. Assuming following info. 
Conversion Efficiency = 20% 
Solar Irradiance is the power per unit area (watt per square metre, W/m2)

## Investigation
1. Hypothesis all Modules of same size. h0 = Mean production rate of each inverter for a given day are all equal. Only based off assumption 'Solar cell area is same for all the modules for all inverters'
2. Calculate solar cell area based on the assumptions and check for consistence across inverters/ days

# Extract performance of plant 1, inverter_1 on 2020-05-15

In [ ]:
# lets pick a day and inverter number for investigation. Say Inverter = 'one', date = '2020:05:16' lets extract data df_gen
#and weather file. 

#extract dataframe for Inverter 'One'
df_1 = df_inv_id.get('one')
df_1.set_index('DATE_TIME', drop=True, append=False, inplace=True, verify_integrity=False)

df_1_day= df_1.loc['2020-05-15 05:30:00': '2020-05-15 19:30:00']

In [ ]:
df_1_day.head()

In [ ]:
## Similarly lets extract info from weather for date and range ['2020-05-15 05:30:00': '2020-05-15 19:30:00']
df_weather.set_index('DATE_TIME', drop=True, append=False, inplace=True, verify_integrity=False)
df_weath_1 = df_weather.loc['2020-05-15 05:30:00': '2020-05-15 19:30:00']

In [ ]:
df_weath_1.head()

In [ ]:
result = pd.concat([df_1_day,df_weath_1 ], axis=1, sort= False)

#timestamp 19:00, 19:15, 19:30 are missing! The DC power is zero and hence all the rows are delated. 
result.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
result.tail()

In [ ]:
sns.heatmap(result.corr(),annot=True)

In [ ]:
sns.pairplot(data=result) 

In [ ]:
sns.jointplot(x='IRRADIATION', y='DC_POWER', data= result, kind = 'reg')
sns.jointplot(x='AMBIENT_TEMPERATURE', y='MODULE_TEMPERATURE', data= result,  kind = 'kde')

# Further cleaning
several columns from result are not important. Example daily_yield, total_yield, Module_temp, AC_power are irrelavent. 
The data shows colinearity and hence they are not included. for future analysis. 

In [ ]:
result = result.drop(['AC_POWER', 'SOURCE_KEY', 'DAILY_YIELD','TOTAL_YIELD', 'MODULE_TEMPERATURE'], axis=1)


In [ ]:
fig = plt.figure();   
ax1 = fig.add_subplot(121);  
ax2 = fig.add_subplot(122);

y =result['DC_POWER']
y1= result['IRRADIATION']

sns.kdeplot(y, shade=True, color="r",gridsize = 100, ax= ax1)
sns.kdeplot(y1, shade=True, color="r",gridsize = 100, ax=ax2)

In [ ]:
y2 = result['AMBIENT_TEMPERATURE']
sns.kdeplot(y2, shade=True, color="r",gridsize = 100)

## Area under the DC_POWER curve & surface area of solar cell. 

In [ ]:
def area_undercurve(df):
    dc_array =df['DC_POWER']
    irrad_array = df['IRRADIATION']
    dc_power = round(sum(dc_array.values*15.0/60.0),4)
    solar_power = round(sum(irrad_array*15.0/60.0),4)
    
    solar_cell_area = round(dc_power/(solar_power*0.2), 4)
    
    return dc_power, solar_power, solar_cell_area


## Extract Information for "Inverter -1"

In [ ]:
# Pseudo code
# 1. filter data for inverter 1 and combine this with weather data. To form one single dataframe
# 2. filter day time dcpower, solar_power and solar_cell_area for specific days with good data. 
# 3. study the variations between days. 
# 4. Hypothesis testing H0: Mean_of_inverter1 = Mean_of_inverter2 Ha means are not equal. 

In [ ]:
## modify the dataframe filter to drop the columns that are not needed. 

df_one_comb = pd.concat([df_1,df_weather], axis=1).reindex(df_1.index)
df_one_comb = df_one_comb.drop(['AC_POWER', 'SOURCE_KEY', 'DAILY_YIELD','TOTAL_YIELD', 'MODULE_TEMPERATURE'], axis=1)

In [ ]:
list_delta = [0,1,2,4,6,8,9,10,11,12,14,15,21,28,29,30,31,32]

start_date = dt.datetime(2020,5,16,5,30,0)
end_date = dt.datetime(2020,5,16,19,30,0)


dates = []
dc_pwr = []
solar_pwr = []
solar_cell_ar= []
inverter = ['one'] * len(list_delta)

for n in list_delta:
    start = start_date + dt.timedelta(days = n)
    end = end_date + dt.timedelta(days = n)
    dates.append(start.date())
    dc_power, solar_power, solar_cell_area = area_undercurve(df_one_comb.loc[start:end])
    dc_pwr.append(dc_power)
    solar_pwr.append(solar_power)
    solar_cell_ar.append(solar_cell_area)
    #print(start,end)
    
data = {'Date': dates, 'DC_power':dc_pwr, 'Solar_power': solar_pwr, 'Solar_cell_area': solar_cell_ar, 'inverter': inverter}
invert_results_df = pd.DataFrame(data= data, columns= ['Date', 'DC_power', 'Solar_power','Solar_cell_area', 'inverter'])

In [ ]:
invert_results_df

In [ ]:
sns.lineplot(x="Date", y="DC_power", data=invert_results_df)

In [ ]:
sns.boxplot(x=invert_results_df["DC_power"])

## Extract Information for "Inverter -2"

In [ ]:
#extract dataframe for Inverter 'One'
df_2 = df_inv_id.get('two')
df_2.set_index('DATE_TIME', drop=True, append=False, inplace=True, verify_integrity=False)

## modify the dataframe filter to drop the columns that are not needed. 
df_two_comb = pd.concat([df_2,df_weather], axis=1).reindex(df_2.index)
df_two_comb = df_two_comb.drop(['AC_POWER', 'SOURCE_KEY', 'DAILY_YIELD','TOTAL_YIELD', 'MODULE_TEMPERATURE'], axis=1)

# dataframe for inverter2
df_two_comb.head()

In [ ]:
list_delta = [0,1,2,4,6,8,10,11,12,14,15,21,28,29,30,31,32]

start_date = dt.datetime(2020,5,16,5,30,0)
end_date = dt.datetime(2020,5,16,19,30,0)


dates = []
dc_pwr = []
solar_pwr = []
solar_cell_ar= []
inverter = ['two'] * len(list_delta)

for n in list_delta:
    start = start_date + dt.timedelta(days = n)
    end = end_date + dt.timedelta(days = n)
    dates.append(start.date())
    dc_power, solar_power, solar_cell_area = area_undercurve(df_two_comb.loc[start:end])
    dc_pwr.append(dc_power)
    solar_pwr.append(solar_power)
    solar_cell_ar.append(solar_cell_area)
    #print(start,end)
    
data_two = {'Date': dates, 'DC_power':dc_pwr, 'Solar_power': solar_pwr, 'Solar_cell_area': solar_cell_ar, 'inverter': inverter}
invert_two_results_df = pd.DataFrame(data= data_two, columns= ['Date', 'DC_power', 'Solar_power','Solar_cell_area', 'inverter'])

invert_two_results_df

## Hypothesis Testing 2sample t test

In [ ]:
from scipy import stats
x1 = invert_results_df['DC_power']
x2 = invert_two_results_df['DC_power']

stats.ttest_ind(x1,x2)

In [ ]:
#Since P value is more than 0.5, statisitically fail to reject Null Hypothesis

## Further Analysis

 "Feel free to copy this notebook and continue analysis. If you wish to!"

1. Compare all other inverters of plant 1 for sample number of days
2. Compare all other inverters of plant 2 for sample number of days
3. Check if weather patterns for plant 1 and plant 2 vary significantly. If yes then the location of plant 2 is different 
from plant 1. 
4. Compare all inverters of plant 1 & 2 for a given day. 